In [14]:
import pandas as pd
import numpy as np
from string import punctuation
from string import digits
from gensim.models import word2vec
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn
import re
from nltk.tokenize import RegexpTokenizer

## Only using reviews

In [16]:
df = pd.read_csv('new_Manhattan.csv')
df = df[df['reviews'] != '[]']
Manhattan_reviews = df['reviews']
Manhattan_is_closed = df['is_closed']

df = pd.read_csv('new_Brooklyn.csv')
df = df[df['reviews'] != '[]']
Brooklyn_reviews = df['reviews']
Brooklyn_is_closed = df['is_closed']

In [18]:
## train_set, test_set = train_test_split(total_reviews, random_state=2020, test_size=0.3)

## list = [re.split('\n| ',line) for line in train_set['review']]
list = [re.split('\n| ',line) for line in Manhattan_reviews]
reviews_divide = [[item.replace(c, '') for c in punctuation for item in line if item != ''] for line in list]

## Train Word2Vec Model
w2v_model = word2vec.Word2Vec(reviews_divide, size=100, window=5, iter=10, workers=1, seed=2018, min_count=1)

## get sentence vectors in train
reviews_ints = []
def get_sentence_vector(review):
    vecs = 0
    i = 0
    for word in review:     
        try:
          i = i + 1
          vecs += w2v_model[word]
        except KeyError:
          continue
    vecs = vecs / i
    return vecs

for i in range(len(reviews_divide)):
    try:
      reviews_ints.append(get_sentence_vector(reviews_divide[i]))
    except:
      print(i)
      print(reviews_divide[i])

## Train SVM,logistic regression,SGD,GradientBoosting, RandomForest Model
svm_clf =  SVC().fit(np.array(reviews_ints), np.array(Manhattan_is_closed))
logistic_clf = LogisticRegression().fit(np.array(reviews_ints), np.array(Manhattan_is_closed))
LinearSVC_clf = LinearSVC().fit(np.array(reviews_ints), np.array(Manhattan_is_closed))
SGD_clf = SGDClassifier().fit(np.array(reviews_ints), np.array(Manhattan_is_closed))
GradientBoosting_clf = GradientBoostingClassifier().fit(np.array(reviews_ints), np.array(Manhattan_is_closed))
RandomForest_clf = RandomForestClassifier().fit(np.array(reviews_ints), np.array(Manhattan_is_closed))

## Performance in Test set
list = [re.split('\n| ',line) for line in Brooklyn_reviews]
reviews_divide = [[item.replace(c, '') for c in punctuation for item in line if item != ''] for line in list]



reviews_ints = []
for i in range(len(reviews_divide)):
    try:
      reviews_ints.append(get_sentence_vector(reviews_divide[i]))
    except:
      print(i)
      print(reviews_divide[i])

svm_predicted = svm_clf.predict(np.array(reviews_ints))
logistic_predicted = logistic_clf.predict(np.array(reviews_ints))
LinearSVC_predicted = LinearSVC_clf.predict(np.array(reviews_ints))
SGD_predicted = SGD_clf.predict(np.array(reviews_ints))
GradientBoosting_predicted = GradientBoosting_clf.predict(np.array(reviews_ints))
RandomForest_predicted = RandomForest_clf.predict(np.array(reviews_ints))



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py

In [19]:
print("svm" ,1 - np.sum((svm_predicted - np.array(Brooklyn_is_closed + 0))**2) / len(svm_predicted))
print("logistic" ,1 - np.sum((logistic_predicted  - np.array(Brooklyn_is_closed + 0))**2) / len(logistic_predicted ))
print("LinearSVC" ,1 - np.sum((LinearSVC_predicted  - np.array(Brooklyn_is_closed + 0))**2) / len(LinearSVC_predicted ))
print("SGD" ,1 - np.sum((SGD_predicted - np.array(Brooklyn_is_closed + 0))**2) / len(SGD_predicted))
print("GradientBoosting" ,1 - np.sum((GradientBoosting_predicted - np.array(Brooklyn_is_closed + 0))**2) / len(GradientBoosting_predicted))
print("RandomForest" ,1 - np.sum((RandomForest_predicted - np.array(Brooklyn_is_closed + 0))**2) / len(RandomForest_predicted))

svm 0.7818791946308725
logistic 0.8322147651006712
LinearSVC 0.8120805369127517
SGD 0.7953020134228188
GradientBoosting 0.8087248322147651
RandomForest 0.7885906040268457


In [21]:
print(sklearn.metrics.roc_auc_score(np.array(Brooklyn_is_closed + 0), svm_predicted))
print(sklearn.metrics.roc_auc_score(np.array(Brooklyn_is_closed + 0), logistic_predicted))
print(sklearn.metrics.roc_auc_score(np.array(Brooklyn_is_closed + 0), LinearSVC_predicted))
print(sklearn.metrics.roc_auc_score(np.array(Brooklyn_is_closed + 0), SGD_predicted))
print(sklearn.metrics.roc_auc_score(np.array(Brooklyn_is_closed + 0), GradientBoosting_predicted))
print(sklearn.metrics.roc_auc_score(np.array(Brooklyn_is_closed + 0), RandomForest_predicted))

0.5
0.659755694948828
0.663519313304721
0.6694288544073952
0.6114559260482006
0.5597556949488279
